In [8]:
import PIL.Image as Image
import os
import pyproj
import math
import cv2
import numpy as np

# Compose satellite images to check overlapping

In [9]:
def image_compose(IMAGES_PATH,IMAGES_FORMAT,IMAGE_SIZE,IMAGE_ROW,IMAGE_COLUMN,IMAGE_SAVE_PATH):
    image_names = [name for name in os.listdir(IMAGES_PATH) for item in IMAGES_FORMAT if
                   os.path.splitext(name)[1] == item]

    if len(image_names) != IMAGE_ROW * IMAGE_COLUMN:
        raise ValueError("Total image amount should be {}".format(IMAGE_ROW*IMAGE_COLUMN))
    
    to_image = Image.new('RGB', (IMAGE_COLUMN * IMAGE_SIZE[0], IMAGE_ROW * IMAGE_SIZE[1])) # create a new image

    for y in range(1, IMAGE_ROW + 1):
        for x in range(1, IMAGE_COLUMN + 1):
            from_image = Image.open(IMAGES_PATH + image_names[IMAGE_COLUMN * (y - 1) + x - 1]).resize(
                IMAGE_SIZE, Image.Resampling.LANCZOS)
            to_image.paste(from_image, ((x - 1) * IMAGE_SIZE[0], (y - 1) * IMAGE_SIZE[1]))
    return to_image.save(IMAGE_SAVE_PATH)

In [10]:
maps_path = './datasets/satellite_imgs/'
save_path = './datasets/sat.png'

IMAGES_FORMAT = ['.png']  # image format
IMAGE_SIZE = (1280, 720)  # single iamge size
IMAGE_ROW = 3
IMAGE_COLUMN = 3

IMAGES_PATH = maps_path
IMAGE_SAVE_PATH = save_path   
image_compose(IMAGES_PATH,IMAGES_FORMAT,IMAGE_SIZE,IMAGE_ROW,IMAGE_COLUMN,IMAGE_SAVE_PATH)

# compute GSD

In [11]:
rows, cols = 3, 3

# load coordinates
with open('./datasets/coordinates.txt') as f:
    lines = f.readlines()
coordinates = np.zeros([rows,cols,2])

for line in lines:
    x, y = int(line[0]), int(line[2])
    coordinates[x,y,0] = float((line.split('\t')[1]).split('=')[1])
    coordinates[x,y,1] = float((line.split('\t')[2]).split('=')[1])

In [12]:
def geo_proj_transform(data, geo2proj=True):
    # convert group of points
    geo_epsg, proj_epsg = "epsg:4326", "epsg:3857"
    if geo2proj:
        transformer = pyproj.Transformer.from_crs(geo_epsg, proj_epsg)
    else:
        transformer = pyproj.Transformer.from_crs(proj_epsg, geo_epsg)
    
    H, W, _ = data.shape
    points = np.zeros([H*W,2])
    for i, point in enumerate(transformer.itransform(data.reshape(-1,2))):
        points[i] = np.array(point)
    return points.reshape(H,W,-1)

In [13]:
projs = geo_proj_transform(coordinates)

In [14]:
H, W, _ = projs.shape

# meter per pixel in East, South direction
GSD = [(projs[-1,-1,0]-projs[0,0,0])/((W-1)*IMAGE_SIZE[0]), (projs[-1,-1,1]-projs[0,0,1])/((H-1)*IMAGE_SIZE[1])] 
GSD

[0.14929908923440963, -0.14923802730692032]

# Calculate target area north, south, east, west and center GPS coordinates

In [15]:
def proj2geo_transform(data, geo2proj=False):
    # convert group of points
    geo_epsg, proj_epsg = "epsg:4326", "epsg:3857"
    if geo2proj:
        transformer = pyproj.Transformer.from_crs(geo_epsg, proj_epsg)
    else:
        transformer = pyproj.Transformer.from_crs(proj_epsg, geo_epsg)

    points = np.zeros([data.shape[0],2])
    for i, point in enumerate(transformer.itransform(data)):
        points[i] = np.array(point)
    return points

In [29]:
corners_proj = np.zeros([3,2])
corners_proj[0] = np.array([projs[0,0,0]-IMAGE_SIZE[0]/2*GSD[0], projs[0,0,1]-IMAGE_SIZE[1]/2*GSD[1]]) # top_left
corners_proj[1] = np.array([projs[-1,-1,0]+IMAGE_SIZE[0]/2*GSD[0], projs[-1,-1,1]+IMAGE_SIZE[1]/2*GSD[1]]) # bottom_right
corners_proj[2] = np.array([corners_proj[0,0]+IMAGE_SIZE[0]*cols/2*GSD[0], corners_proj[0,1]+IMAGE_SIZE[1]*rows/2*GSD[1]]) # center

corners_gps = proj2geo_transform(corners_proj)
print('Target area north GPS: {}'.format(corners_gps[0,0]))
print('Target area south GPS: {}'.format(corners_gps[1,0]))
print('Target area east GPS: {}'.format(corners_gps[1,1]))
print('Target area west GPS: {}'.format(corners_gps[0,1]))
print('Target area center GPS: {}'.format(corners_gps[2]))
print('Target area has width of {} meters'.format(abs(IMAGE_SIZE[1]*rows*GSD[1])))
print('Target area has height of {} meters'.format(abs(IMAGE_SIZE[0]*cols*GSD[0])))

Target area north GPS: 40.00050846915017
Target area south GPS: 39.99829017146142
Target area east GPS: -83.0139618955573
Target area west GPS: -83.01911201346185
Target area center GPS: [ 39.99939933 -83.01653695]
Target area has width of 322.35413898294786 meters
Target area has height of 573.308502660133 meters
